<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">

*このノートブックには、Jake VanderPlas による [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) からの抜粋が含まれています。コンテンツは利用可能です [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do).*

※テキストは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で、コードは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で公開しています。このコンテンツが役立つと思われる場合は、[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode) による作業のサポートを検討してください!*

<!--ナビゲーション-->
< [Introduction to NumPy](02.00-Introduction-to-NumPy.ipynb) | [Introduction to NumPy](02.00-Introduction-to-NumPy.ipynb) | [Introduction to NumPy](02.00-Introduction-to-NumPy.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/02.01-Understanding-Data-Types.ipynb"><img align="left" src=" https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>


# Python のデータ型について

効果的なデータ主導の科学と計算には、データがどのように保存され、操作されるかを理解する必要があります。
このセクションでは、データの配列が Python 言語自体でどのように処理されるか、および NumPy がこれをどのように改善するかについて概説し、対比します。
この違いを理解することは、本書の残りの部分の多くの内容を理解するための基本です。

Python のユーザーは、その使いやすさに魅了されることが多く、その 1 つに動的型付けがあります。
C や Java などの静的に型付けされた言語では、各変数を明示的に宣言する必要がありますが、Python などの動的に型付けされた言語では、この仕様がスキップされます。たとえば、C では、次のように特定の操作を指定できます。

```C
/* C code */
int result = 0;
for(int i=0; i<100; i++){
    result += i;
}
```

Python では、同等の操作を次のように書くことができます。

```python
# Python code
result = 0
for i in range(100):
    result += i
```

主な違いに注意してください。C では各変数のデータ型が明示的に宣言されますが、Python では型が動的に推論されます。これは、たとえば、あらゆる種類のデータをあらゆる変数に割り当てることができることを意味します。

```python
# Python code
x = 4
x = "four"
```

ここでは ``x`` の内容を整数から文字列に切り替えました。 C で同じことを行うと、(コンパイラの設定によっては) コンパイル エラーやその他の意図しない結果が発生します。

```C
/* C code */
int x = 4;
x = "four";  // FAILS
```

この種の柔軟性は、Python やその他の動的型付け言語を便利で使いやすいものにする 1 つの要素です。
これがどのように機能するかを理解することは、Python でデータを効率的かつ効果的に分析するための重要な学習要素です。
しかし、この型の柔軟性が示しているのは、Python 変数は単なる値以上のものであるという事実です。また、値の型に関する追加情報も含まれています。これについては、以降のセクションで詳しく説明します。

## Python 整数は単なる整数以上のものです

標準の Python 実装は C で記述されています。
これは、すべての Python オブジェクトが単に巧妙に偽装された C 構造体であり、その値だけでなく他の情報も含まれていることを意味します。たとえば、Python で ``x = 10000`` のような整数を定義する場合、``x`` は単なる「生の」整数ではありません。実際には、複数の値を含む複合 C 構造体へのポインターです。
Python 3.4 のソース コードに目を通すと、整数 (long) 型の定義が事実上次のようになっていることがわかります (C マクロを展開すると)。

```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```

Python 3.4 の 1 つの整数には、実際には次の 4 つの部分が含まれます。

- ``ob_refcnt``、Python がメモリの割り当てと割り当て解除を静かに処理するのに役立つ参照カウント
- 変数の型をエンコードする ``ob_type``
- 次のデータメンバーのサイズを指定する ``ob_size``
- ``ob_digit`` には、Python 変数が表すと予想される実際の整数値が含まれています。

これは、次の図に示すように、C などのコンパイル済み言語での整数と比較して、Python での整数の格納にはオーバーヘッドがあることを意味します。

![Integer Memory Layout](figures/cint_vs_pyint.png)

ここで ``PyObject_HEAD`` は、参照カウント、型コード、および前述のその他の部分を含む構造体の一部です。

ここでの違いに注意してください: C 整数は本質的に、バイトが整数値をエンコードするメモリ内の位置のラベルです。
Python 整数は、整数値を含むバイトを含む、すべての Python オブジェクト情報を含むメモリ内の位置へのポインターです。
この Python 整数構造の追加情報により、Python を自由かつ動的にコーディングできます。
ただし、Python 型のこの追加情報には代償が伴います。これは、これらのオブジェクトの多くを組み合わせた構造で特に顕著になります。

## Python リストは単なるリストではありません

多くの Python オブジェクトを保持する Python データ構造を使用するとどうなるかを考えてみましょう。
Python の標準の変更可能な複数要素コンテナーはリストです。
次のように整数のリストを作成できます。

In [1]:
L = list(range(10))
L

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [2]:
type(L[0])

int

または、同様に、文字列のリスト:

In [3]:
L2 = [str(c) for c in L]
L2

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [4]:
type(L2[0])

str

Python の動的型付けにより、異種のリストを作成することさえできます。

In [5]:
L3 = [True, "2", 3.0, 4]
[type(item) for item in L3]

[bool, str, float, int]

ただし、この柔軟性には代償が伴います。これらの柔軟な型を許可するには、リスト内の各項目に独自の型情報、参照カウント、およびその他の情報が含まれている必要があります。つまり、各項目は完全な Python オブジェクトです。
すべての変数が同じ型であるという特殊なケースでは、この情報の多くは冗長です。固定型の配列にデータを格納する方がはるかに効率的です。
動的型リストと固定型 (NumPy スタイル) 配列の違いを次の図に示します。

![Array Memory Layout](figures/array_vs_list.png)

実装レベルでは、配列には基本的に、1 つの連続したデータ ブロックへの単一のポインターが含まれます。
一方、Python リストには、ポインターのブロックへのポインターが含まれており、それぞれが、前に見た Python 整数のような完全な Python オブジェクトを指しています。
繰り返しになりますが、リストの利点は柔軟性です。各リスト要素はデータと型情報の両方を含む完全な構造であるため、リストには任意の型のデータを入力できます。
固定型の NumPy スタイルの配列にはこのような柔軟性がありませんが、データの保存と操作に関してははるかに効率的です。

## Python の固定型配列

Python には、効率的な固定型データ バッファーにデータを格納するためのさまざまなオプションが用意されています。
組み込みの ``array`` モジュール (Python 3.3 以降で利用可能) を使用して、均一な型の密な配列を作成できます。

In [6]:
import array
L = list(range(10))
A = array.array('i', L)
A

array('i', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

ここで ``'i'`` は、内容が整数であることを示す型コードです。

しかし、もっと便利なのは NumPy パッケージの ``ndarray`` オブジェクトです。
Python の ``array`` オブジェクトは配列ベースのデータの効率的なストレージを提供しますが、NumPy はそのデータに対するこの効率的な *操作* を追加します。
これらの操作については、後のセクションで説明します。ここでは、NumPy 配列を作成するいくつかの方法を示します。

エイリアス ``np`` の下で、標準の NumPy インポートから始めます。

In [7]:
import numpy as np

## Python リストから配列を作成する

まず、 np.array を使用して Python リストから配列を作成できます。

In [8]:
# integer array:
np.array([1, 4, 2, 5, 3])

array([1, 4, 2, 5, 3])

Python のリストとは異なり、NumPy はすべて同じ型を含む配列に制約されることに注意してください。
タイプが一致しない場合、NumPy は可能であればアップキャストします (ここでは、整数は浮動小数点にアップキャストされます)。

In [9]:
np.array([3.14, 4, 2, 3])

array([ 3.14,  4.  ,  2.  ,  3.  ])

結果の配列のデータ型を明示的に設定したい場合は、 ``dtype`` キーワードを使用できます:

In [10]:
np.array([1, 2, 3, 4], dtype='float32')

array([ 1.,  2.,  3.,  4.], dtype=float32)

最後に、Python リストとは異なり、NumPy 配列は明示的に多次元にすることができます。リストのリストを使用して多次元配列を初期化する 1 つの方法を次に示します。

In [11]:
# nested lists result in multi-dimensional arrays
np.array([range(i, i + 3) for i in [2, 4, 6]])

array([[2, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

内側のリストは、結果の 2 次元配列の行として扱われます。

## 最初から配列を作成する

特に大きな配列の場合、NumPy に組み込まれているルーチンを使用してゼロから配列を作成する方が効率的です。
以下にいくつかの例を示します。

In [12]:
# Create a length-10 integer array filled with zeros
np.zeros(10, dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [13]:
# Create a 3x5 floating-point array filled with ones
np.ones((3, 5), dtype=float)

array([[ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.]])

In [14]:
# Create a 3x5 array filled with 3.14
np.full((3, 5), 3.14)

array([[ 3.14,  3.14,  3.14,  3.14,  3.14],
       [ 3.14,  3.14,  3.14,  3.14,  3.14],
       [ 3.14,  3.14,  3.14,  3.14,  3.14]])

In [15]:
# Create an array filled with a linear sequence
# Starting at 0, ending at 20, stepping by 2
# (this is similar to the built-in range() function)
np.arange(0, 20, 2)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [16]:
# Create an array of five values evenly spaced between 0 and 1
np.linspace(0, 1, 5)

array([ 0.  ,  0.25,  0.5 ,  0.75,  1.  ])

In [17]:
# Create a 3x3 array of uniformly distributed
# random values between 0 and 1
np.random.random((3, 3))

array([[ 0.99844933,  0.52183819,  0.22421193],
       [ 0.08007488,  0.45429293,  0.20941444],
       [ 0.14360941,  0.96910973,  0.946117  ]])

In [18]:
# Create a 3x3 array of normally distributed random values
# with mean 0 and standard deviation 1
np.random.normal(0, 1, (3, 3))

array([[ 1.51772646,  0.39614948, -0.10634696],
       [ 0.25671348,  0.00732722,  0.37783601],
       [ 0.68446945,  0.15926039, -0.70744073]])

In [19]:
# Create a 3x3 array of random integers in the interval [0, 10)
np.random.randint(0, 10, (3, 3))

array([[2, 3, 4],
       [5, 7, 8],
       [0, 5, 0]])

In [20]:
# Create a 3x3 identity matrix
np.eye(3)

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [21]:
# Create an uninitialized array of three integers
# The values will be whatever happens to already exist at that memory location
np.empty(3)

array([ 1.,  1.,  1.])

## NumPy 標準データ型

NumPy 配列には単一の型の値が含まれているため、それらの型とその制限について詳細に理解しておくことが重要です。
NumPy は C で構築されているため、型は C、Fortran、およびその他の関連言語のユーザーになじみがあります。

標準の NumPy データ型を次の表に示します。
配列を構築するときは、文字列を使用して指定できることに注意してください。

```python
np.zeros(10, dtype='int16')
```

または、関連する NumPy オブジェクトを使用します。

```python
np.zeros(10, dtype=np.int16)
```

| |データ型 |説明 |
|---------------|-------------|
| | ``bool_`` |バイトとして格納されるブール値 (True または False) |
| | ``int_`` |デフォルトの整数型 (C の ``long`` と同じ; 通常は ``int64`` または ``int32``)|
| | ``intc`` | C の ``int`` (通常は ``int32`` または ``int64``) と同じです|
| | ``intp`` |インデックス付けに使用される整数 (C の ``ssize_t`` と同じ; 通常は ``int32`` または ``int64``)|
| | ``int8`` |バイト (-128 ～ 127)|
| | ``int16`` |整数 (-32768 ～ 32767)|
| | ``int32`` |整数 (-2147483648 ～ 2147483647)|
| | ``int64`` |整数 (-9223372036854775808 ～ 9223372036854775807)|
| | ``uint8`` |符号なし整数 (0 ～ 255)|
| | uint16 |符号なし整数 (0 ～ 65535)|
| | uint32 |符号なし整数 (0 ～ 4294967295)|
| | uint64 |符号なし整数 (0 ～ 18446744073709551615)|
| | ``float_`` | ``float64`` の省略形。|
| | ``float16`` |半精度浮動小数点: 符号ビット、指数 5 ビット、仮数 10 ビット|
| | ``float32`` |単精度浮動小数点: 符号ビット、指数 8 ビット、仮数 23 ビット |
| | ``float64`` |倍精度浮動小数点: 符号ビット、11 ビット指数、52 ビット仮数|
| |複合体 | ``complex128`` の省略形。|
| | ``complex64`` | 2 つの 32 ビット float で表される複素数 |
| | ``complex128``| 2 つの 64 ビット float で表される複素数 |

ビッグ エンディアン番号またはリトル エンディアン番号を指定するなど、より高度な型指定が可能です。詳細については、[NumPy documentation](http://numpy.org/) を参照してください。
NumPy は、[Structured Data: NumPy's Structured Arrays](02.09-Structured-Data-NumPy.ipynb) でカバーされる複合データ型もサポートします。

<!--ナビゲーション-->
< [Introduction to NumPy](02.00-Introduction-to-NumPy.ipynb) | [Introduction to NumPy](02.00-Introduction-to-NumPy.ipynb) | [Introduction to NumPy](02.00-Introduction-to-NumPy.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/02.01-Understanding-Data-Types.ipynb"><img align="left" src=" https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>
